<a href="https://colab.research.google.com/github/eumoas/Data-Engineer/blob/main/SPOTIFY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SPOTIFY -  utilizando os conhecimentos em limpeza e tratamento de dados e os comandos da ferramenta Pyspark, faça os tratamentos necessários para que os Datasets fiquem prontos para análise. Em seguida, pense em 5 insights interessantes para cada um dos arquivos e com as funções de seleção, filtragem, etc...do pyspark mostre esses insights.

In [ ]:
!pip install pyspark

In [ ]:
!pip install gcsfs

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F
import pandas as pd
from google.cloud import storage
import os
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
serviceAccount = '/content/speedy-district-339220-e8cc21176e65.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [14]:
client = storage.Client()
bucket = client.get_bucket('spotifyteste')
bucket.blob('spotify.csv')
path = 'gs://spotifyteste/spotify.csv'

df_pandas = pd.read_csv(path)

In [15]:
df_pandas.head()

,Unnamed: 0,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,0,Boulevard of Broken Dreams,73,262333,0.005520000000000001kg,0.496mol/L,0.682,2.94e-05,8.0,0.0589,-4.095,1,0.0294,167.06,4,0.474
1,1,In The End,66,216933,0.0103kg,0.542mol/L,0.853,0.0,3.0,0.108,-6.407,0,0.0498,105.256,4,0.370
2,2,Seven Nation Army,76,231733,0.00817kg,0.737mol/L,0.463,0.447,0.0,0.255,-7.8279999999999985,1,0.0792,123.881,4,0.324
3,3,By The Way,74,216933,0.0264kg,0.451mol/L,0.97,0.00355,0.0,0.102,-4.938,1,0.107,122.444,4,0.198
4,4,How You Remind Me,56,223826,0.000954kg,0.447mol/L,0.7659999999999999,0.0,10.0,0.113,-5.065,1,0.0313,172.011,4,0.574


In [16]:
spark = (
    SparkSession.builder
                .master('local')
                .appName('aulaSpark')
                .config('spark.ui.port', '4050')
                .getOrCreate()
)

In [17]:
df_spark = spark.createDataFrame(df_pandas)

In [18]:
df_spark.show()

+----------+--------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+----+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|Unnamed: 0|           song_name|song_popularity|song_duration_ms|        acousticness|        danceability|            energy|    instrumentalness| key|           liveness|           loudness|audio_mode|         speechiness|  tempo|time_signature|     audio_valence|
+----------+--------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+----+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|         0|Boulevard of Brok...|             73|          262333|0.005520000000000...|          0.496mol/L|             0.682|            2.94e-05| 8.0|             0.0589|             -4.095|   

In [20]:
df2 = (df_spark.toDF(*('ID', 'nome_da_música', 'popularidade_da_música', 'duração_da_música (ms)', 'acústica', 'dançabilidade', 'energia', 'instrumentalidade', 'chave', 'vivacidade', 'volume',
              'mode_de_áudio',  'fala', 'tempo', 'tempo_de_assinatura',  'valencia_de_áudio' )) 
)

In [21]:
df2.show()


+---+--------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+-----+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+
| ID|      nome_da_música|popularidade_da_música|duração_da_música (ms)|            acústica|       dançabilidade|           energia|   instrumentalidade|chave|         vivacidade|             volume|mode_de_áudio|                fala|  tempo|tempo_de_assinatura| valencia_de_áudio|
+---+--------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+-----+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+
|  0|Boulevard of Brok...|                    73|                262333|0.005520000000000...|          0.496mol/L|             0.682|            2.94e-

In [27]:


df2.select(F.col("nome_da_música"), F.col("popularidade_da_música"), F.col("acústica")).filter(F.col("tempo_de_assinatura") >= "4").show (20)

+--------------------+----------------------+--------------------+
|      nome_da_música|popularidade_da_música|            acústica|
+--------------------+----------------------+--------------------+
|Boulevard of Brok...|                    73|0.005520000000000...|
|          In The End|                    66|            0.0103kg|
|   Seven Nation Army|                    76|           0.00817kg|
|          By The Way|                    74|            0.0264kg|
|   How You Remind Me|                    56|          0.000954kg|
|    Bring Me To Life|                    80|           0.00895kg|
|         Last Resort|                    81|          0.000504kg|
|Are You Gonna Be ...|                    76|           0.00148kg|
|      Mr. Brightside|                    80|           0.00108kg|
|         Sex on Fire|                    81|           0.00172kg|
|          The Middle|                    78|            0.0424kg|
|                Numb|                    63|            0.004

In [28]:
df2.filter(F.col("popularidade_da_música") == "77").show(5)

+---+--------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+-----+----------+------+-------------+------+-------+-------------------+-----------------+
| ID|      nome_da_música|popularidade_da_música|duração_da_música (ms)|            acústica|       dançabilidade|           energia|   instrumentalidade|chave|vivacidade|volume|mode_de_áudio|  fala|  tempo|tempo_de_assinatura|valencia_de_áudio|
+---+--------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+-----+----------+------+-------------+------+-------+-------------------+-----------------+
| 15|         Take Me Out|                    77|                237026|          0.000423kg|          0.278mol/L|0.6759999999999999|0.000899000000000...|  9.0|     0.136|-8.821|            1|0.0371|104.545|                  4|            0.494|
| 18|I Write Sin

In [29]:
df2.filter(F.col("popularidade_da_música") >= "81").show(20)

+---+--------------------+----------------------+----------------------+--------------------+--------------------+------------------+-----------------+-----+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+
| ID|      nome_da_música|popularidade_da_música|duração_da_música (ms)|            acústica|       dançabilidade|           energia|instrumentalidade|chave|         vivacidade|             volume|mode_de_áudio|                fala|  tempo|tempo_de_assinatura| valencia_de_áudio|
+---+--------------------+----------------------+----------------------+--------------------+--------------------+------------------+-----------------+-----+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+
|  6|         Last Resort|                    81|                199893|          0.000504kg|          0.581mol/L|             0.887|          0.00111|  4.0|   

In [30]:
df2.filter(F.col("popularidade_da_música") >= "99").show(20)

+-----+--------------------+----------------------+----------------------+----------+-------------+-------+-----------------+-----+----------+------+-------------+-----+-------+-------------------+-----------------+
|   ID|      nome_da_música|popularidade_da_música|duração_da_música (ms)|  acústica|dançabilidade|energia|instrumentalidade|chave|vivacidade|volume|mode_de_áudio| fala|  tempo|tempo_de_assinatura|valencia_de_áudio|
+-----+--------------------+----------------------+----------------------+----------+-------------+-------+-----------------+-----+----------+------+-------------+-----+-------+-------------------+-----------------+
| 1229|I Love It (& Lil ...|                    99|                127946|  0.0114kg|   0.901mol/L|  0.522|              0.0|  2.0|     0.259|-8.304|            1| 0.33|104.053|                  4|            0.329|
| 1757| Party In The U.S.A.|               nao_sei|  0.8220000000000001kg|0.519mol/L|         0.36|    0.0|               10|0.177|    -

Insight 1 = Música que começam com a letra A

In [41]:

df2.filter(F.col("nome_da_música").startswith("A")).show(10)


+---+--------------------+----------------------+----------------------+----------+--------------------+-------+--------------------+-----+-------------------+-------+-------------+------+------------------+-------------------+------------------+
| ID|      nome_da_música|popularidade_da_música|duração_da_música (ms)|  acústica|       dançabilidade|energia|   instrumentalidade|chave|         vivacidade| volume|mode_de_áudio|  fala|             tempo|tempo_de_assinatura| valencia_de_áudio|
+---+--------------------+----------------------+----------------------+----------+--------------------+-------+--------------------+-----+-------------------+-------+-------------+------+------------------+-------------------+------------------+
|  7|Are You Gonna Be ...|                    76|                213800| 0.00148kg|          0.613mol/L|  0.953|            0.000582|  2.0|              0.152| -3.435|            1|0.0855|           105.046|                  4|             0.537|
| 27|      A

Insight 2 = Mostrar música que tenham energia maior que 0.517

In [43]:
from pyspark.sql.window import Window

In [48]:
df2.select(F.col("popularidade_da_música")).filter(F.col("energia") > 0.517).show()

+----------------------+
|popularidade_da_música|
+----------------------+
|                    73|
|                    66|
|                    74|
|                    56|
|                    80|
|                    81|
|                    76|
|                    80|
|                    81|
|                    78|
|                    63|
|                    75|
|                    81|
|                    69|
|                    77|
|                    71|
|                    62|
|                    77|
|                    79|
|                    69|
+----------------------+
only showing top 20 rows



Insight 3 = Mostrar todos os dados apenas das músicas que contenham a palavra love


In [50]:
df2.filter(F.col('nome_da_música').startswith('love')).show(truncate=False)

+-----+--------------------+----------------------+----------------------+--------+------------------------+-------------------+-----------------+-----+----------+------------------+-------------+------+-------+-------------------+------------------+
|ID   |nome_da_música      |popularidade_da_música|duração_da_música (ms)|acústica|dançabilidade           |energia            |instrumentalidade|chave|vivacidade|volume            |mode_de_áudio|fala  |tempo  |tempo_de_assinatura|valencia_de_áudio |
+-----+--------------------+----------------------+----------------------+--------+------------------------+-------------------+-----------------+-----+----------+------------------+-------------+------+-------+-------------------+------------------+
|4732 |lovely (with Khalid)|90                    |200185                |0.934kg |0.35100000000000003mol/L|0.29600000000000004|0.0              |4.0  |0.095     |-10.109           |0            |0.0333|115.284|4                  |0.12            

Insight 4 = Mostrar a soma do tempo de todas as músicas



In [54]:

df2.agg(F.sum('tempo_de_assinatura')).show(5)

+------------------------+
|sum(tempo_de_assinatura)|
+------------------------+
|          2.8000745587E9|
+------------------------+



Insight 5 = Mostrar em uma nova coluna o seguinte dado: Caso a música tenha popularidade maior que 90, inserir música Tophits, caso contrário, colocar músicas comuns. 





In [56]:

df2 = ( df2.withColumn('Spotify', 
                     F.when(F.col('popularidade_da_música') > 90, F.lit('TopHits'))
                     .otherwise(F.lit('Música comum')))

).show()

+---+--------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+-----+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+------------+
| ID|      nome_da_música|popularidade_da_música|duração_da_música (ms)|            acústica|       dançabilidade|           energia|   instrumentalidade|chave|         vivacidade|             volume|mode_de_áudio|                fala|  tempo|tempo_de_assinatura| valencia_de_áudio|     Spotify|
+---+--------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+-----+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+------------+
|  0|Boulevard of Brok...|                    73|                262333|0.005520000000000...|          0.496mol/